In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('up_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/lucknow-to-aligarh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF75ECBEEA2+31554]
	(No symbol) [0x00007FF75EC37ED9]
	(No symbol) [0x00007FF75EAF872A]
	(No symbol) [0x00007FF75EB48434]
	(No symbol) [0x00007FF75EB4853C]
	(No symbol) [0x00007FF75EB8F6A7]
	(No symbol) [0x00007FF75EB6D06F]
	(No symbol) [0x00007FF75EB8C977]
	(No symbol) [0x00007FF75EB6CDD3]
	(No symbol) [0x00007FF75EB3A33B]
	(No symbol) [0x00007FF75EB3AED1]
	GetHandleVerifier [0x00007FF75EFC8B1D+3217341]
	GetHandleVerifier [0x00007FF75F015AE3+3532675]
	GetHandleVerifier [0x00007FF75F00B0E0+3489152]
	GetHandleVerifier [0x00007FF75ED6E776+750614]
	(No symbol) [0x00007FF75EC4375F]
	(No symbol) [0x00007FF75EC3EB14]
	(No symbol) [0x00007FF75EC3ECA2]
	(No symbol) [0x00007FF75EC2E16F]
	BaseThreadInitThunk [0x00007FFE5633257D+29]
	RtlUserThreadStart [0x00007FFE572CAF28+40]



NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - PLB0002,Shatabdi AC Seater 2+2,00:25,05h 35m,06:00,4.1,INR 598,25 Seats available
1,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - STD0072,Janrath AC Seater 2+2,00:40,05h 05m,05:45,3.3,INR 598,26 Seats available
2,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - KSB0043,Shatabdi AC Seater 2+2,06:00,05h 30m,11:30,2.6,INR 598,36 Seats available
3,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - BLY0150,Janrath AC Seater 2+2,07:05,04h 40m,11:45,3.3,INR 598,32 Seats available
4,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - BLY0145,Rajdhani Non AC Seater 2+3,07:30,05h 00m,12:30,3.3,INR 448,52 Seats available
...,...,...,...,...,...,...,...,...,...,...
2021,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Shri Krishna Bus Service,A/C Seater / Sleeper (2+1),23:01,05h 31m,04:32,2.4,INR 699,40 Seats available
2022,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,ROYAL SAFARI INDIA TRAVELS,A/C Semi Sleeper (2+2),22:30,07h 30m,06:00,0,1079,36 Seats available
2023,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,ROYAL SAFARI INDIA TRAVELS,Bharat Benz A/C Seater /Sleeper (2+1),22:30,07h 30m,06:00,0,INR 499,24 Seats available
2024,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Kainchi dham express,A/C Seater Tempo Traveller (2+1),21:00,07h 30m,04:30,0,INR 999,16 Seats available


In [3]:
df.tail(50)

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
1976,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,UPSRTC - RMP0018,Ordinary Non AC Seater 2+3,19:30,07h 30m,03:00,1.5,INR 478,51 Seats available
1977,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,R S YADAV SMART BUS PRIVATE LIMITED,A/C Seater / Sleeper (2+1),22:00,07h 00m,05:00,4.8,569,40 Seats available
1978,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Fasttrack Travels,VE A/C Seater / Sleeper (2+2),22:30,06h 45m,05:15,4.5,539,36 Seats available
1979,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,International Tourist Centre,VE A/C Seater / Sleeper (2+1),22:25,06h 40m,05:05,4.5,INR 549,35 Seats available
1980,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Laxmi holidays,Bharat Benz A/C Sleeper (2+1),23:00,06h 00m,05:00,4.4,664,36 Seats available
1981,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Laxmi holidays,VE A/C Seater / Sleeper (2+1),22:30,06h 00m,04:30,4.3,474,35 Seats available
1982,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Laxmi holidays,VE A/C Sleeper (2+1),23:30,05h 45m,05:15,4.4,740,25 Seats available
1983,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Blueworld tourist private limited,A/C Seater / Sleeper (2+1),23:25,06h 20m,05:45,4.6,INR 599,23 Seats available
1984,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,International Tourist Centre,VE A/C Seater / Sleeper (2+1),22:15,06h 45m,05:00,4.5,INR 549,32 Seats available
1985,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Fasttrack Travels,A/C Seater/Sleeper (2+1),21:20,08h 10m,05:30,4.5,539,31 Seats available
